In [66]:
import pandas as pd
import re

def preprocess(x):
    x = str(x) #轉str
    x = re.sub('[\u4e00-\u9fa5]', '', x) # 去除中文
    x = re.sub(r'[^\w\s]','',x) # 去除標點符號
    x = x.replace('\n', '').replace('\r', '').replace('\t', '') # 去除換行符號
    return str.strip(x)

# 載入公司寶典
公司寶典 = pd.read_excel('..\data\寶典\台塑出口公司名稱及統編.xlsx')[['代號','公司英文名稱']]
公司寶典['公司英文名稱'] = 公司寶典['公司英文名稱'].apply(lambda x:preprocess(x))
print(公司寶典.shape)
keep_idx = []
for i in 公司寶典.index:
    if len(str(公司寶典.loc[i,'代號']))>=2:
        keep_idx.append(i)
公司寶典 = 公司寶典.loc[keep_idx]
公司寶典

(28, 2)


,代號,公司英文名稱
4,AF,FORMOSA AUTOMOBILE SALES CORPORATION
5,AH,FORMOSA ASAHI SPANDEX CO LTD
6,AL,FORMOSA DAIKIN ADVANCED CHEMICALS CO LTD
7,BM,FORMOSA BIOMEDICAL TECHNOLOGY CORPORATION
8,BN,HONG JING RESOURCES CO LTD
9,GF,CHANG GUNG MEDICAL TECHNOLOGY CO LTD
10,IF,IDEMITSU FORMOSA SPECIALTY CHEMICALS CORPORATION
11,J7,INTEPLAST TAIWAN CORPORATION
12,RS,FORMOSA IDEMITSU PETROCHEMICAL CORP
13,RW,NAN YA PHOTONICS INCORPORATION


In [67]:
# 載入訓練資料
train_df_公司 = pd.read_csv('../data/preprocess_for_SQUAD_公司.csv')[['EXPNO','Y_label']].dropna(axis=0)
train_df_公司.columns = ['代號','公司英文名稱']
print(train_df_公司.shape)
train_df_公司.head()

(8209, 2)


,代號,公司英文名稱
3,11,FORMOSA PLASTICS CORPORATION
4,27,NAN YA PLASTICS CORPORATION
5,18,FORMOSA PLASTICS CORPORATION
7,25,NAN YA PLASTICS CORPORATION
8,25,NAN YA PLASTICS CORPORATION


In [76]:
train_df_公司[train_df_公司['公司英文名稱']=='FORMOSA PLASTICS CORPORATION'].drop_duplicates(subset=['代號'])

,代號,公司英文名稱
3,11,FORMOSA PLASTICS CORPORATION
5,18,FORMOSA PLASTICS CORPORATION
61,1P,FORMOSA PLASTICS CORPORATION
350,1A,FORMOSA PLASTICS CORPORATION
1779,12,FORMOSA PLASTICS CORPORATION
3400,14,FORMOSA PLASTICS CORPORATION


In [68]:
# 產入產品寶典
# 讀取台塑網提供之(寶典)
df1 = pd.read_excel('../data/寶典/台塑企業_ 產品寶典20210303.xlsx',engine='openpyxl')[['公司代號','公司事業部門','品名']]
df2 = pd.read_excel('../data/寶典/寶典.v3.台塑網.20210901.xlsx',engine='openpyxl')[['CODIV','DIVNM','ITEMNM']]
df2 = df2.rename(columns={'ITEMNM':'品名','DIVNM':'公司事業部門','CODIV':'公司代號'})
df3 = pd.read_excel('../data/寶典/寶典.v4.20211001.xlsx',engine='openpyxl')
df3 = df3.rename(columns={'ITEMNM':'品名','DIVNM':'公司事業部門','CODIV':'公司代號'})
df4 = pd.read_excel('../data/寶典/寶典.v5.20211006.xlsx',engine='openpyxl')
df4 = df4.rename(columns={'ITEMNM':'品名','DIVNM':'公司事業部門','CODIV':'公司代號'})
df = df1.append(df2).append(df3).append(df4) # 合併所有寶典
代號2部門 = dict(zip(df['公司代號'],df['公司事業部門']))

In [69]:
from tqdm import tqdm_notebook as tqdm
df = 公司寶典.append(train_df_公司).drop_duplicates(subset=['公司英文名稱']).reset_index(drop=True)
df['公司事業部門'] = 'not find'
error = []
for i in tqdm(df.index):
    try:
        df.loc[i,'公司事業部門'] = 代號2部門[df.loc[i,'代號']]
    except:
         error.append(df.loc[i,'代號'])
error

<ipython-input-69-ec71c39a02ad>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(df.index):


['AF',
 'AH',
 'AL',
 'BM',
 'BN',
 'GF',
 'IF',
 'J7',
 'RS',
 'RW',
 'RZ',
 'XU',
 'XZ',
 'BQ',
 'L1',
 'RC',
 'RP',
 'RA',
 'RV',
 'TN',
 'TR',
 'BW',
 'AH']

In [70]:
df

,代號,公司英文名稱,公司事業部門
0,AF,FORMOSA AUTOMOBILE SALES CORPORATION,not find
1,AH,FORMOSA ASAHI SPANDEX CO LTD,not find
2,AL,FORMOSA DAIKIN ADVANCED CHEMICALS CO LTD,not find
3,BM,FORMOSA BIOMEDICAL TECHNOLOGY CORPORATION,not find
4,BN,HONG JING RESOURCES CO LTD,not find
5,GF,CHANG GUNG MEDICAL TECHNOLOGY CO LTD,not find
6,IF,IDEMITSU FORMOSA SPECIALTY CHEMICALS CORPORATION,not find
7,J7,INTEPLAST TAIWAN CORPORATION,not find
8,RS,FORMOSA IDEMITSU PETROCHEMICAL CORP,not find
9,RW,NAN YA PHOTONICS INCORPORATION,not find


In [71]:
df['尾綴'] = df['公司英文名稱'].apply(lambda x:x.split(' ')[-1])
df

,代號,公司英文名稱,公司事業部門,尾綴
0,AF,FORMOSA AUTOMOBILE SALES CORPORATION,not find,CORPORATION
1,AH,FORMOSA ASAHI SPANDEX CO LTD,not find,LTD
2,AL,FORMOSA DAIKIN ADVANCED CHEMICALS CO LTD,not find,LTD
3,BM,FORMOSA BIOMEDICAL TECHNOLOGY CORPORATION,not find,CORPORATION
4,BN,HONG JING RESOURCES CO LTD,not find,LTD
5,GF,CHANG GUNG MEDICAL TECHNOLOGY CO LTD,not find,LTD
6,IF,IDEMITSU FORMOSA SPECIALTY CHEMICALS CORPORATION,not find,CORPORATION
7,J7,INTEPLAST TAIWAN CORPORATION,not find,CORPORATION
8,RS,FORMOSA IDEMITSU PETROCHEMICAL CORP,not find,CORP
9,RW,NAN YA PHOTONICS INCORPORATION,not find,INCORPORATION


In [72]:
df.shape

(60, 4)

In [73]:
df.to_csv('../data/寶典/公司寶典加尾綴(擴充版).csv')